In [1]:
import numpy as np
import csv
import os
import pandas as pd

In [2]:
class Logger:

    def __init__(self, ruta_csv, overwrite=True):
        self.ruta_csv = ruta_csv
        self.dataset = None

        # Crear archivo con cabecera
        if overwrite or not os.path.exists(self.ruta_csv):
            with open(self.ruta_csv, mode='w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(["Dataset", "Algoritmo", "Semilla", "Costo", "Evaluaciones", "Solucion"])
        else:
            self.data = pd.read_csv(self.ruta_csv)

    def set_dataset(self, dataset):
        self.dataset = dataset

    def log(self, algoritmo, semilla, costo, evaluaciones, solucion):
        if not self.dataset:
            raise Exception("Dataset not set!")
        fila = [
            self.dataset,
            algoritmo,
            semilla,
            costo,
            evaluaciones,
            solucion
        ]

        with open(self.ruta_csv, mode='a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(fila)


####################################### FUNCTIONS #######################################


def load_mats(path, verbose=False):
    """
    Leer archivos .dat
        1ra matriz -> distancias
        2da matriz -> flujos
    """
    try:
        with open(path, "r") as file:
            lines = file.readlines()
            try:
                mat_size = int(lines[0].strip())
                if (verbose): print(f"mat_size={mat_size}")

                n_mats = int((len(lines)-1) / mat_size)
                if (verbose): print(f"nmats={n_mats}")
                mats = np.zeros((n_mats, mat_size, mat_size), dtype=np.int32)
                for idx_mat in range(n_mats):
                    for i in range(1, mat_size+1):
                        elements = list(filter(None, lines[idx_mat*mat_size + i].strip().split(' ')))
                        if elements:
                            if len(elements) != mat_size:
                                print(f"Matriz irregular [{len(elements)} != {mat_size}] en {path} (linea {idx_mat*mat_size + i})\nDetail:\n{elements}")
                                return dict()
                            for j in range(len(elements)):
                                if elements[j]:
                                    mats[idx_mat, i-1, j] = np.int32(elements[j])
                return {"nombre" : path, "distancia" : mats[0, :, :], "flujo" : mats[1, :, :]}
            except ValueError:
                print("ValueError")
            except:
                print(f"Error al parsear {path}")
    except:
        print(f"Archivo {path} no existe")

    return dict()

def save(path, solucion, valor, iteraciones):
    try:
        file = open(path, "w")

        file.write(f"{valor} {iteraciones}\n")
        for el in solucion:
            file.write(f"{el} ")
        file.write("\n")
    except:
        print(f"Unable to write file {path}")

# Funcion objetivo de kevin
def funcion_objetivo(solucion, matD, matF):
    permutado = matF[solucion, :][:, solucion]
    return np.sum(np.multiply(matD, permutado), dtype=np.int32)

def funcion_objetivo_original_QAP(solucion, matD, matF):
    result = 0
    n = matD.shape[0]

    for i in range(n):
        for j in range(n):
                if i != j:
                    result += matD[i, j] * matF[solucion[i] , solucion[j] ]
    return result

def delta_pocha(S, d, f, r, s):
    n = len(S)
    delta_val = 0
    for k in range(n):
        if k == r or k == s:
            continue
        delta_val += (
            d[r][k] * (f[S[r]][S[k]] - f[S[s]][S[k]]) +
            d[k][r] * (f[S[k]][S[r]] - f[S[k]][S[s]]) +
            d[s][k] * (f[S[s]][S[k]] - f[S[r]][S[k]]) +
            d[k][s] * (f[S[k]][S[s]] - f[S[k]][S[r]])
        )
    delta_val += (
        d[r][s] * (f[S[r]][S[s]] - f[S[s]][S[r]]) +
        d[s][r] * (f[S[s]][S[r]] - f[S[r]][S[s]])
    )
    return delta_val

def delta(S, d, f, r, s):
    S = np.asarray(S)
    n = len(S)

    k = np.array([k for k in range(n) if k != r and k != s])

    # Preextraer ubicaciones
    Sr, Ss = S[r], S[s]
    Sk = S[k]

    # Vectorizados
    term1 = d[r, k] * (f[Sr, Sk] - f[Ss, Sk])
    term2 = d[k, r] * (f[Sk, Sr] - f[Sk, Ss])
    term3 = d[s, k] * (f[Ss, Sk] - f[Sr, Sk])
    term4 = d[k, s] * (f[Sk, Ss] - f[Sk, Sr])

    # Suma total de los cuatro vectores
    delta_val = np.sum(term1 + term2 + term3 + term4, dtype=np.int32)

    # Términos fuera del bucle
    delta_val += d[r][s] * (f[Sr][Ss] - f[Ss][Sr]) + d[s][r] * (f[Ss][Sr] - f[Sr][Ss])

    return delta_val

def operador_sublista_aleatoria(solucion, s : int):
    """
    Operador de generacion de vecino -> Sublista Aleatoria de Tamaño Fijo:
    este proceso consiste en generar aleatoriamente dos posiciones
     que determinen una sublista de tamaño s, analizar las asignaciones
    efectuadas entre ambas y reasignarlas aleatoriamente. Se considerarán
    sublistas cíclicas y, por tanto, la posición inicial de la sublista
    no siempre será inferior a la posición final.
    """
    n = len(solucion)
    inicio = np.random.randint(n)

    # Crear los índices cíclicos de la sublista
    indices = [(inicio + i) % n for i in range(s)]

    # Extraer y mezclar la sublista
    sublista = [solucion[i] for i in indices]
    shuffled = sublista.copy()
    while shuffled == sublista:
        np.random.shuffle(shuffled)

    # Reinsertar la sublista mezclada
    nueva_solucion = solucion.copy()
    for idx, val in zip(indices, shuffled):
        nueva_solucion[idx] = val

    return nueva_solucion

def vector_to_str(vct : list) -> str:
    s = " "
    for e in vct:
        s = s + str(e) + " "

    return s

def test_funcion_objetivo():
    prueba_optima = [3, 4, 1, 2]
    tai25_optima = [3, 14, 9, 8, 12, 4, 24, 18, 6, 2, 16, 5, 17, 19, 15, 1, 21, 22, 7, 10, 20, 23, 13, 11, 0]
    sko90_optima = [85, 26, 8, 87, 75, 40, 51, 37, 78, 7, 47, 58, 74, 83, 15, 43, 39, 60, 57, 24, 49, 29, 48,
                 79, 56, 65, 62, 73, 42, 81, 70, 68, 14, 2, 6, 18, 77, 12, 16, 52, 22, 35, 11, 36, 27, 50, 5,
                 13, 76, 28, 33, 55, 69, 64, 82, 46, 53, 3, 0, 1, 30, 59, 9, 88, 89, 25, 80, 72, 20, 63, 21,
                 44, 54, 67, 84, 34, 31, 61, 86, 41, 10, 17, 45, 19, 32, 71, 38, 66, 23, 4]

    tai150_optima = [83, 62, 114, 18, 57, 64, 50, 104, 102, 5, 29, 26, 82, 122, 98, 24, 146, 30, 124, 47, 46,
                115, 75, 143, 103, 81, 112, 60, 108, 110, 131, 92, 53, 87, 140, 127, 109, 36, 145, 63, 129,
                67, 123, 45, 120, 72, 91, 113, 73, 12, 21, 74, 137, 141, 118, 147, 125, 68, 41, 17, 52, 132,
                130, 15, 88, 86, 136, 38, 119, 23, 20, 70, 1, 8, 90, 3, 96, 89, 148, 25, 59, 65, 121, 99, 55,
                94, 126, 135, 19, 95, 76, 56, 48, 101, 138, 133, 34, 71, 28, 111, 31, 77, 40, 79, 128, 93, 35,
                7, 13, 32, 39, 11, 105, 97, 33, 142, 149, 27, 116, 6, 66, 100, 107, 80, 106, 69, 4, 0, 54, 2,
                117, 51, 9, 84, 78, 14, 22, 61, 42, 43, 37, 134, 44, 58, 144, 139, 49, 85, 16, 10]


In [13]:
#################################### BUSQUEDA GREEDY ####################################

def busqueda_greedy(matD, matF):
    """
    Construye una solución inicial para el QAP usando una heurística greedy probabilística.

    :param flujo: Matriz de flujo entre unidades (numpy array n x n).
    :param distancia: Matriz de distancia entre localizaciones (numpy array n x n).
    :return: solucion.
    """
    n = matD.shape[0]
    idx_localizaciones_no_asignadas = np.arange(0, n, dtype=np.uint8)
    idx_unidades_no_asignadas = np.arange(0, n, dtype=np.uint8)

    # Calcular potenciales
    promedio_distancias = np.sum(matD, axis=1)  # Distancia total por localización
    promedio_flujos = np.sum(matF, axis=1) # Flujo total por unidad

    solucion = np.zeros(n, dtype=np.uint8)

    idx_mejores_distancias = sorted(idx_localizaciones_no_asignadas, key=lambda i: promedio_distancias[i])
    idx_mejores_flujos = sorted(idx_unidades_no_asignadas, key=lambda i: promedio_flujos[i], reverse=True)

    while idx_localizaciones_no_asignadas.size != 0:
        # Selección de los mejores candidatos
        idx_localizacion_mejor = idx_mejores_distancias.pop(0)
        idx_unidad_mejor = idx_mejores_flujos.pop(0)

        #solucion[idx_localizacion_mejor] = idx_unidad_mejor
        # Segun ChatGPT es asi
        solucion[idx_unidad_mejor] = idx_localizacion_mejor

        idx_localizaciones_no_asignadas = np.delete(idx_localizaciones_no_asignadas, np.where(idx_localizaciones_no_asignadas == idx_localizacion_mejor))
        idx_unidades_no_asignadas = np.delete(idx_unidades_no_asignadas, np.where(idx_unidades_no_asignadas == idx_unidad_mejor))

    return solucion

################################### ALGORITMO GENETICO ###################################

def seleccion_torneo(poblacion, fitness, k=3):
    indices = np.random.choice(len(poblacion), k, replace=False)
    mejor = min(indices, key=lambda i: fitness[i])
    return poblacion[mejor]

def cruzar(padre1, padre2):
    """ Cruce por posición (PMX simplificado para permutaciones) """
    n = len(padre1)
    punto = np.random.randint(1, n - 1)
    hijo = [-1] * n
    hijo[:punto] = padre1[:punto]

    for i in range(punto, n):
        for gen in padre2:
            if gen not in hijo:
                hijo[i] = gen
                break
    return np.array(hijo)

def mutacion(solucion, tasa=0.1):
    """ Intercambia dos posiciones aleatoriamente con cierta probabilidad """
    if np.random.rand() < tasa:
        i, j = np.random.choice(len(solucion), 2, replace=False)
        solucion[i], solucion[j] = solucion[j], solucion[i]
    return solucion

def algoritmo_genetico(matD, matF, n_generaciones=100, tam_poblacion=20, tasa_mutacion=0.1):
    n = matD.shape[0]
    poblacion = [np.random.permutation(n) for _ in range(tam_poblacion)]
    fitness = [funcion_objetivo(ind, matD, matF) for ind in poblacion]

    mejor_solucion = poblacion[np.argmin(fitness)]
    mejor_valor = min(fitness)
    evaluaciones = 0

    for gen in range(n_generaciones):
        nueva_poblacion = []

        for _ in range(tam_poblacion):
            padre1 = seleccion_torneo(poblacion, fitness)
            padre2 = seleccion_torneo(poblacion, fitness)
            hijo = cruzar(padre1, padre2)
            hijo = mutacion(hijo, tasa_mutacion)
            valor = funcion_objetivo(hijo, matD, matF)
            evaluaciones += 1
            nueva_poblacion.append(hijo)

            if valor < mejor_valor:
                mejor_solucion = hijo
                mejor_valor = valor

        poblacion = nueva_poblacion
        fitness = [funcion_objetivo(ind, matD, matF) for ind in poblacion]


    return mejor_solucion, mejor_valor, evaluaciones

In [17]:
prueba = load_mats("prueba.dat")
tai25 = load_mats("tai25b.dat")
sko90 = load_mats("sko90b.dat")
tai150 = load_mats("tai150b.dat")
main_logger = Logger("practica3.csv")


for d in [tai25]:

    matD = d['distancia']
    matF = d['flujo']
    main_logger.set_dataset(d['nombre'])

    solucion = busqueda_greedy(matD, matF)
    valor = funcion_objetivo(solucion, matD, matF)
    main_logger.log("Greedy", "N/A", valor, 0, vector_to_str(solucion))

    for seed in [42, 420, 4200, 42000, 80987]:
        np.random.seed(seed)
        solucion, valor, evaluaciones = algoritmo_genetico(matD, matF, n_generaciones=100, tam_poblacion=20, tasa_mutacion=0.1)
        main_logger.log("Genetico", seed, valor, evaluaciones, vector_to_str(solucion))
